In [1]:
import pandas as pd
import numpy as np

#read in data from csv from folder ../data
crime_df = pd.read_csv('../data/cf_apprissAppend_va2_10_17_2022.csv')
medical_df = pd.read_csv('../data/CF_medicalData_v2.csv')
PII_df = pd.read_csv('../data/dev_sample_HeartTable_10_17_2022.csv')

C:\Users\cayde\AppData\Local\Temp\ipykernel_14300\739831198.py:5: DtypeWarning: Columns (26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_df = pd.read_csv('../data/cf_apprissAppend_va2_10_17_2022.csv')
C:\Users\cayde\AppData\Local\Temp\ipykernel_14300\739831198.py:6: DtypeWarning: Columns (16,17,18,19,20,21,22,23,34,36,49,50,51,52,53,54,55,56,57,58,59,60,62,63,64,74,75,76,78,79,80,85,87,88,89,90,91,92,93,97,98,99,108,123,124) have mixed types. Specify dtype option on import or set low_memory=False.
  medical_df = pd.read_csv('../data/CF_medicalData_v2.csv')


In [2]:
# cut down the dataframe medical_df to only the columns we need 'source_id', 'ENDate'
medical_df = medical_df[['source_id', 'ENDate', 'MiddleName']]
# join the dataframes PII_df and medical_df left on source_id
PII_med_df = pd.merge(PII_df, medical_df, on='source_id', how='left')
PII_med_df = PII_med_df[['source_id', 'FIRST_NAME', 'MiddleName', 'LAST_NAME', 'MASTER_DOB', 'ENDate']]



In [3]:
# cut down crime_df to only the columns we need 'source_id', 'SenderId' 'PersonFirstName' 'PersonMiddleName' 'PersonLastName' 'BookingDate'
crime_df = crime_df[['PK','source_id', 'SenderId', 'PersonFirstName', 'PersonMiddleName', 'PersonLastName', 'BookingDate', 'PersonYearOfBirth']]
# rename the column "pk" to booking_event_id
crime_df = crime_df.rename(columns={'PK': 'booking_event_id'})


In [4]:
# join the dataframes PII_med_df and crime_df right on source_id
Crime_PII_df = pd.merge(PII_med_df, crime_df, on='source_id', how='right')

In [5]:
# for the columns PersonFirstName, PersonMiddleName, PersonLastName remove the first and last character
Crime_PII_df['PersonFirstName'] = Crime_PII_df['PersonFirstName'].str[1:-1]
Crime_PII_df['PersonMiddleName'] = Crime_PII_df['PersonMiddleName'].str[1:-1]
Crime_PII_df['PersonLastName'] = Crime_PII_df['PersonLastName'].str[1:-1]
Crime_PII_df['PersonYearOfBirth'] = Crime_PII_df['PersonYearOfBirth'].str[1:-1]
# this will get rid of all of the [] in the columns

In [6]:
#import regular expressions
import re
# # check to see whether each source_id is associated with a unique sender_id
# Crime_PII_df.groupby('source_id').SenderId.nunique().value_counts()

# For all rows where ENDate is not null, check whether BookingDate occurs after ENDate
Crime_PII_df['DiedBeforeBookingCharge'] = pd.to_datetime(Crime_PII_df['BookingDate']) > pd.to_datetime(Crime_PII_df['ENDate'])

def YOB_match(row):
    if type(row['PersonYearOfBirth']) == str:
        pass
    else:
        return False
    # use regular expression to find if 'FIRST_NAME' column in Crime_PII_df contains 'PersonFirstName' column in Crime_PII_df
    if re.search(row['PersonYearOfBirth'], row['MASTER_DOB']):
        return True
    else:
        return False

Crime_PII_df['YOB_match'] = Crime_PII_df.apply(YOB_match, axis=1)

In [7]:
# cast all strings to all lower case
Crime_PII_df['FIRST_NAME'] = Crime_PII_df['FIRST_NAME'].str.lower()
Crime_PII_df['PersonFirstName'] = Crime_PII_df['PersonFirstName'].str.lower()
Crime_PII_df['LAST_NAME'] = Crime_PII_df['LAST_NAME'].str.lower()
Crime_PII_df['PersonLastName'] = Crime_PII_df['PersonLastName'].str.lower()
Crime_PII_df['PersonMiddleName'] = Crime_PII_df['PersonMiddleName'].str.lower()
Crime_PII_df['MiddleName'] = Crime_PII_df['MiddleName'].str.lower()

In [8]:
# from matts review file adding 
# 2. Suffixes(jr, sr, ii, iii, and iv) 
# 4. Last Names with a hypthen or apostrophe 

# remove suffixes from last name
# create a list of suffixes to remove ( this list can be updated with more suffixes also need to look out for the case where multiple sufixxes are present)
[', i', ', ii', ', iii', ', iv', 'jr.', 'sr.']

# remove suffixes from last name
Crime_PII_df['LAST_NAME'] = Crime_PII_df['LAST_NAME'].str.replace(', iii', '')
Crime_PII_df['LAST_NAME'] = Crime_PII_df['LAST_NAME'].str.replace(', ii', '')
Crime_PII_df['LAST_NAME'] = Crime_PII_df['LAST_NAME'].str.replace(', i', '')
Crime_PII_df['LAST_NAME'] = Crime_PII_df['LAST_NAME'].str.replace(', iv', '')
Crime_PII_df['LAST_NAME'] = Crime_PII_df['LAST_NAME'].str.replace('jr.', '')
Crime_PII_df['LAST_NAME'] = Crime_PII_df['LAST_NAME'].str.replace('sr.', '')

# remove hypthen from last name and middle name and first name
Crime_PII_df['LAST_NAME'] = Crime_PII_df['LAST_NAME'].str.replace('-', '')
Crime_PII_df['MiddleName'] = Crime_PII_df['MiddleName'].str.replace('-', '')
Crime_PII_df['FIRST_NAME'] = Crime_PII_df['FIRST_NAME'].str.replace('-', '')

# remove apostrophe from last name and middle name and first name
Crime_PII_df['LAST_NAME'] = Crime_PII_df['LAST_NAME'].str.replace("'", '')
Crime_PII_df['MiddleName'] = Crime_PII_df['MiddleName'].str.replace("'", '')
Crime_PII_df['FIRST_NAME'] = Crime_PII_df['FIRST_NAME'].str.replace("'", '')

C:\Users\cayde\AppData\Local\Temp\ipykernel_14300\2136133465.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  Crime_PII_df['LAST_NAME'] = Crime_PII_df['LAST_NAME'].str.replace('jr.', '')
C:\Users\cayde\AppData\Local\Temp\ipykernel_14300\2136133465.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  Crime_PII_df['LAST_NAME'] = Crime_PII_df['LAST_NAME'].str.replace('sr.', '')


In [9]:
#import regular expressions
import re

# create a function that returns a boolean value for whether the first name in the crime data matches the first name in the PII data
def first_name_match(row):
    if type(row['PersonFirstName']) == str:
        pass
    else:
        return False
    # use regular expression to find if 'FIRST_NAME' column in Crime_PII_df contains 'PersonFirstName' column in Crime_PII_df
    if re.search(row['FIRST_NAME'], row['PersonFirstName']):
        return True
    else:
        return False

# create a function that returns a boolean value for whether the last name in the crime data matches the last name in the PII data
def last_name_match(row):
    if type(row['PersonLastName']) == str:
        pass
    else:
        return False
    # use regular expression to find if 'LAST_NAME' column in Crime_PII_df contains 'PersonLastName' column in Crime_PII_df
    if re.search(row['LAST_NAME'], row['PersonLastName']):
        return True
    else:
        return False

# create a function that returns a boolean value for whether the middle name in the crime data matches the middle name in the PII data
def middle_name_match(row):
    # convert middle_name to a string
    middle_name = str(row['MiddleName'])
    # if middle_name is null, return False
    if middle_name == 'nan':
        return 'no_middle_name_on_medical_record'
    # if row['PersonMiddleName'] is an empty string, return False
    if row['PersonMiddleName'] == '':
        return 'no_middle_name_on_appriss_record'
    # check if row['PersonMiddleName'] can be searched with re.search without throwing an error
    if type(row['PersonMiddleName']) == str:
        pass
    else:
        return 'False'
    # use regular expression to find if 'PersonMiddleName' column in Crime_PII_df contains 'PersonMiddleName' column in Crime_PII_df
    if re.search(middle_name, row['PersonMiddleName']):
        return 'True'
    else:
        return 'False'

Crime_PII_df['FN_match'] = Crime_PII_df.apply(first_name_match, axis=1)
Crime_PII_df['LN_match'] = Crime_PII_df.apply(last_name_match, axis=1)
Crime_PII_df['MN_match'] = Crime_PII_df.apply(middle_name_match, axis=1)

In [10]:
# from matts review file adding 
# 3. Names That a Phonetic Similiar that do not meet match critieria

# FuzzyWuzzy is a Python library that provides a set of string matching algorithms for fuzzy matching (i.e., matching strings that are similar, but not necessarily identical). It is used in data cleaning and data preprocessing tasks to identify and correct errors/inconsistencies in data.
# It uses the Levenshtein distance equation: A measure of the difference between two strings. From what I can understand, it the minimum number of character edits(deleting, substuting, adding) required to turn one string into the other.
# The scores that it gives are from 0-100. With 100 being a very similiar or exact name.
# using 80 as the threshold for a match

# import fuzzywuzzy
from fuzzywuzzy import fuzz

# create a function that uses fuzzywuzzy to compare if names are close to matching
def fuzzy_match(x, y, threshold=93):
    ratio = fuzz.ratio(x, y)
    if ratio >= threshold:
        return True
    else:
        return False

#apply lambda functions to first name middle name and last name to see if we have any matches create a column if this is true
Crime_PII_df['fuzzy_match_first_name'] = Crime_PII_df.apply(lambda x: fuzzy_match(x['PersonFirstName'], x['FIRST_NAME']), axis=1)
# create a second column that is the score or fuzzy match
Crime_PII_df['fuzzy_match_first_name_score'] = Crime_PII_df.apply(lambda x: fuzz.ratio(x['PersonFirstName'], x['FIRST_NAME']), axis=1)
Crime_PII_df['fuzzy_match_last_name'] = Crime_PII_df.apply(lambda x: fuzzy_match(x['PersonLastName'], x['LAST_NAME']), axis=1)
Crime_PII_df['fuzzy_match_last_name_score'] = Crime_PII_df.apply(lambda x: fuzz.ratio(x['PersonLastName'], x['LAST_NAME']), axis=1)


In [11]:
from nicknames import NickNamer
nn = NickNamer()

# create a function that checks to see if a name is a nickname
def is_nickname(name, poss_nickname):
    nicks = nn.nicknames_of(name)
    if poss_nickname in nicks:
        return True
    else:
        return False

# apply lambda functions to check if first name or last name are nicknames
Crime_PII_df['is_nickname_first_name'] = Crime_PII_df.apply(lambda x: is_nickname(x['FIRST_NAME'], x['PersonFirstName']), axis=1)
Crime_PII_df['is_nickname_last_name'] = Crime_PII_df.apply(lambda x: is_nickname(x['LAST_NAME'], x['PersonLastName']), axis=1)

# for demonstration purposes
# create a column that is a list of nicknames using lambda function
Crime_PII_df['poss_nicknames_fn'] = Crime_PII_df.apply(lambda x: nn.nicknames_of(x['FIRST_NAME']), axis=1)
Crime_PII_df['poss_nicknames_ln'] = Crime_PII_df.apply(lambda x: nn.nicknames_of(x['LAST_NAME']), axis=1)




In [12]:
# different combinations of first, middle, and last name matches
Crime_PII_df['FN_LN_match'] = Crime_PII_df['FN_match'] & Crime_PII_df['LN_match']
Crime_PII_df['FN_MN_match'] = Crime_PII_df['FN_match'] & Crime_PII_df['MN_match']
Crime_PII_df['LN_MN_match'] = Crime_PII_df['LN_match'] & Crime_PII_df['MN_match']
Crime_PII_df['FN_LN_MN_match'] = Crime_PII_df['FN_match'] & Crime_PII_df['LN_match'] & Crime_PII_df['MN_match']

In [13]:
# turn PersonLastName into a list
Crime_PII_df['PersonLastName_list'] = Crime_PII_df['PersonLastName'].str.split(',')

# print person last name list for row 33
Crime_PII_df['PersonLastName_list'][33]

['aaron brown', ' brown']

In [14]:

def has_multi_names(row):
    if len(row['PersonLastName_list']) > 1:
        return True
    else:
        return False

def count_multi_names(row):
    if len(row['PersonLastName_list']) >= 1:
    #write a switch statment that checks to see if the list of names is between 1-5, 6-10, 11-20, 20+
        if len(row['PersonLastName_list']) <= 5:
            return '1-5'
        elif len(row['PersonLastName_list']) <= 10:
            return '6-10'
        elif len(row['PersonLastName_list']) <= 20:
            return '11-20'
        else:
            return '20+'

Crime_PII_df['has_mult_names'] = Crime_PII_df.apply(has_multi_names, axis=1)
Crime_PII_df['count_mult_names'] = Crime_PII_df.apply(count_multi_names, axis=1)


In [11]:
# write dataframe to csv
Crime_PII_df.to_csv('../data/out/Confidence_match_crimedata.csv')

In [15]:
import pandas as pd
import numpy as np

# write dataframe to xlsx
Crime_PII_df.to_excel('../data/out/Confidence_match_crimedata_v2.xlsx', sheet_name='Sheet1')